In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import os
import glob
import tqdm

import sys
import umap

import scipy.linalg
import pickle
import re

In [3]:
DO_WHITENING = True
REG_PARAM = 1e-1
MAX_POINTS = 15000

# Auxiliary functions

In [4]:
def load_single_cells(allele):
    single_cells = []
    file_handlers = [] # "52649_e15_1/1.jpg"
    for key in tqdm.tqdm(rdf[rdf["Treatment"].str.contains(allele, regex=True)].index):
        data = np.load(rdf.loc[key, "filename"])
        data = data["features"] #[0:32,...]
        single_cells.append(data)
        file_handlers += [
            rdf.loc[key, "filename"].replace(".npz","/") + str(i) + ".jpg"
            for i in range(data.shape[0])
        ]
    if len(single_cells) > 0:
        all_cells = np.concatenate(single_cells)
        P = np.where(~np.isnan(np.sum(all_cells, axis=1)))
        return all_cells[P[0],: ], file_handlers
    else:
        return None, None

In [5]:
def sample(features, image_names, size):
    N = features.shape[0]
    R = np.arange(N)
    np.random.shuffle(R)
    
    R = R[0:size]
    sample_features = features[R]
    sample_images = [image_names[x] for x in R]
    
    plates = [re.match(r"^(.+)_(.+)_(.)", x).groups()[0] for x in sample_images]
    
    return sample_features, sample_images, plates

In [6]:
def prepare_allele(allele_id, mean_ctl, normalizer):
    # Load single cell data of wt and mut
    allele_features, allele_imgs = load_single_cells(allele_id)
    print(allele_id, allele_features.shape)
    allele_features = normalizer.normalize(allele_features, allele_imgs)
    
    allele_data = {
        "id": allele_id,
        "features": allele_features,
        #"whitened": allele_wh,
        "images": allele_imgs
    }
    return allele_data

In [7]:
def compute_umap_projection(wt_id, mutant_ids, controls, ctlimgs, mean_ctl, normalizer, embedding="features"):
    # Load data for wild type and mutants    
    wild_type_data = prepare_allele(wt_id, mean_ctl, normalizer)
    mutants_data = {mut_id: prepare_allele(mut_id, mean_ctl, normalizer) for mut_id in mutant_ids}
    
    # Subsample for fast debugging
    num_ctl = len(ctlimgs)
    wild_type_data[embedding], wild_type_data["images"], _ = sample(wild_type_data[embedding], wild_type_data["images"], num_ctl)
    for k in mutant_ids:
        mutants_data[k][embedding], mutants_data[k]["images"], _ = sample(mutants_data[k][embedding], mutants_data[k]["images"], num_ctl)
    
    # UMAP projection for all mutants of this wild type
    features = np.concatenate( 
        [controls, 
         wild_type_data[embedding]] + 
        [mutants_data[k][embedding] for k in mutant_ids] 
    )
    print("Computing UMAP for", features.shape[0], "data points")   
    Y = umap.UMAP(n_neighbors=15).fit_transform(features)
    
    # Save computations
    with open("outputs/single-cells/" + wt_id + ".pkl", "wb") as out:
        pickle.dump({
            "wild_type_data":wild_type_data, 
            "mutant_ids": mutant_ids,
            "mutants_data":mutants_data, 
            "controls": {embedding: controls},
            "ctlimgs":ctlimgs,
            "graph_scores":0,
            "Y": Y
        }, out)

# Load single-cell data

In [8]:
root_dir = "/dgx1nas1/storage/data/jarevalo/luad/dp_project/"
feature_dir = root_dir + "outputs/efn_pretrained/"
files = [f for f in glob.glob(feature_dir + '/**/*.npz', recursive=True) if f.endswith(".npz")]
len(files)

55292

In [9]:
df = pd.read_csv(root_dir + "inputs/metadata/index.csv")
alleles = pd.read_csv("/raid/data/luad/dp-project/inputs/metadata/filtered/TAOE014015/Allele.csv")
os.makedirs("outputs/single-cells/", exist_ok=True)

In [10]:
if os.path.exists("file_data_frame.csv"):
    fdf = pd.read_csv("file_data_frame.csv")
else:
    fdf = pd.DataFrame(columns=["Metadata_Plate", "Metadata_Well", "Metadata_Site", "filename"])
    i = 0
    for f in tqdm.tqdm(files):
        key = f.replace(".npz","").split("features/")[-1]
        plate, ws = key.split("/")
        well, site = ws.split("_")
        fdf.loc[i] = {"Metadata_Plate": int(plate), "Metadata_Well":well, "Metadata_Site":int(site), "filename":f}
        i += 1
    fdf.to_csv("file_data_frame.csv")

In [11]:
fdf.Metadata_Site = pd.to_numeric(fdf.Metadata_Site)
fdf.Metadata_Plate = pd.to_numeric(fdf.Metadata_Plate)

In [12]:
rdf0 = pd.merge(df, fdf, on=["Metadata_Plate", "Metadata_Well", "Metadata_Site"])
rdf = pd.merge(rdf0, alleles, how="left", left_on="Allele", right_on="ID")

In [13]:
len(rdf["Treatment"].unique())

594

# Population level profiles

In [14]:
## Read aggregated CellProfiler features
morphology = pd.read_csv("/raid/data/luad/dp-project/inputs/metadata/morphology.csv")

## Read well level profiles created with the profiling/profiles-weakly-supervised.ipnb notebook:
profiles = pd.read_parquet("/dgx1nas1/storage/data/jarevalo/luad/dp_project/efn_pretrained_profiles.parquet")

treatments = morphology["Metadata_broad_sample"].unique()
print("Treatments:", len(treatments))
FF = 23 # First feature in data frame

mdata = pd.merge(morphology[morphology.columns[0:FF]], profiles, on=["Metadata_Plate", "Metadata_Well"])
mdata = mdata.reset_index(drop=True)
mdata.loc[mdata["Allele"].isnull(), "Allele"] = "EMPTY"

profiles = np.asarray(mdata[mdata.columns[FF+1:-2]])

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Treatments: 306


In [15]:
# Load controls
aggregated_controls = np.asarray(mdata.loc[mdata["Allele"] == "EMPTY", mdata.columns[FF:]])
agg_mean_ctl = np.mean(aggregated_controls, axis=0)

# Make sample of single-cell controls

In [16]:
class AllControlsNormalizer(object):
    def __init__(self, sc_controls, ctlimgs):
        self.MEAN = np.mean(sc_controls)
        self.STD = np.std(sc_controls)
        
    def normalize(self, features, images):
        return (features - self.MEAN) / self.STD

In [17]:
class SCWhiteningNormalizer(object):
    def __init__(self, sc_controls):
        # Whitening transform on population level data
        self.mu = np.mean(sc_controls, axis=0)
        self.whitening_transform(sc_controls - self.mu, REG_PARAM)
        print(self.mu.shape, self.W.shape)
        
    def whitening_transform(self, X, lambda_, rotate=True):
        C = (1/X.shape[0]) * np.dot(X.T, X)
        s, V = scipy.linalg.eigh(C)
        D = np.diag( 1. / np.sqrt(s + lambda_) )
        W = np.dot(V, D)
        if rotate:
            W = np.dot(W, V.T)
        self.W = W

    def normalize(self, X, images):
        return np.dot(X - self.mu, self.W)

In [ ]:
sc_controls, ctlimgs = load_single_cells("NA@EMPTY")

100%|██████████| 2880/2880 [05:32<00:00,  8.65it/s]


In [ ]:
# Whiten single-cell controls
if DO_WHITENING:
    normalizer = SCWhiteningNormalizer(sc_controls)
else:
    normalizer = AllControlsNormalizer(sc_controls, ctlimgs)
    
controls_sample, ctlimgs_sample, _ = sample(sc_controls, ctlimgs, MAX_POINTS)
controls_sample = normalizer.normalize(controls_sample, ctlimgs_sample)

# Compute UMAP projections

In [ ]:
# Identify wild type alleles
wild_type_ids = [al for al in rdf["Treatment"].unique() if al.endswith("_WT.c")]

# Analyze each mutant
for wt_id in wild_type_ids:
    #if wt_id.find("CCND1") == -1: continue # Uncomment to test on a single variant
    wt = wt_id.split("@")[1].replace("_WT.c","")
    mutant_ids = [al for al in rdf["Treatment"].unique() if al.find(wt) != -1 and not al.endswith("_WT.c")]

    %time compute_umap_projection(wt_id, mutant_ids, controls_sample, ctlimgs_sample, agg_mean_ctl, normalizer)